In [2]:
import numpy as np
import pandas as pd


C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
df = pd.read_csv('insurance.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [12]:
df_encoded = pd.get_dummies(df, columns=['smoker'])
df_encoded = pd.get_dummies(df_encoded, columns=['region'])
df_encoded = pd.get_dummies(df_encoded, columns=['sex'])

In [13]:
df_encoded.head()

,age,bmi,children,charges,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,sex_female,sex_male
0,19,27.900,0,16884.92400,False,True,False,False,False,True,True,False
1,18,33.770,1,1725.55230,True,False,False,False,True,False,False,True
2,28,33.000,3,4449.46200,True,False,False,False,True,False,False,True
3,33,22.705,0,21984.47061,True,False,False,True,False,False,False,True
4,32,28.880,0,3866.85520,True,False,False,True,False,False,False,True


In [9]:
from sklearn.linear_model import LinearRegression

In [14]:
model = LinearRegression()
model.fit(df_encoded.drop('charges', axis = 'columns'), df_encoded.charges)

LinearRegression()

In [15]:
model.coef_

array([   256.85635254,    339.19345361,    475.50054515, -11924.26727096,
        11924.26727096,    587.00923503,    234.0453356 ,   -448.01281436,
         -373.04175627,     65.6571797 ,    -65.6571797 ])

In [18]:
model.intercept_

-666.9377199366281

In [30]:
age, bmi, children, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest, sex_female, sex_male = [0]*11
print('Nhap do tuoi cua ban:')
age = float(input())
print('Nhap chi so bmi: ')
bmi = float(input())
print('Nhap so con cua ban: ')
children = float(input())
print('Nhap gioi tinh cua ban (0:female, 1:male): ')
sex = float(input())
print('Co hut thuoc khong (0: NO, 1:Yes): ')
smoker = float(input())
print('Noi o cua ban (0: northeast, 1: northwest, 2: southeast, 3: southwest): ')
region = float(input())

if sex == 0: 
    sex_female = 1 
else: 
    sex_male = 1
    
if smoker == 0: 
    smoker_no = 1 
else: 
    smoker_yes = 1
    
if region == 0: 
    region_northeast = 1
elif region == 1: 
    region_northwest = 1
elif region == 2: 
    region_southeast = 1
else: 
    region_southwest = 1

vecto = (age, bmi, children, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest, sex_female, sex_male)

chargers = model.intercept_ + sum(vecto[i] * model.coef_[i] for i in range(len(vecto)))
print('Luong cua ban la: ',chargers)

Nhap do tuoi cua ban:
20
Nhap chi so bmi: 
120
Nhap so con cua ban: 
0
Nhap gioi tinh cua ban (0:female, 1:male): 
1
Co hut thuoc khong (0: NO, 1:Yes): 
1
Noi o cua ban (0: northeast, 1: northwest, 2: southeast, 3: southwest): 
1
Luong cua ban la:  57266.059190973356


### Việc sử dụng cả bộ dữ liệu khiến cho không thể biết độ hiệu quả của mô hình do vậy cần phải thực hiện lại

#### Chia mô hình thành 2 bộ train và test

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X, y = df_encoded.drop('charges', axis = 'columns'), df_encoded.charges

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [38]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

LinearRegression()

In [47]:
y_pred = linear_model.predict(X_test)

In [48]:
from sklearn.metrics import r2_score

In [49]:
r2 = r2_score(y_test, y_pred)

In [50]:
print(r2)

0.7835929767120723


 các biến độc lập giải thích được 78.36% sự biến thiên của biến phụ thuộc

In [55]:
from sklearn.metrics import mean_absolute_percentage_error

In [56]:
mape = mean_absolute_percentage_error(y_test, y_pred)
mape

0.46888255979146687

tỷ lệ phần trăm lỗi trung bình giữa giá trị dự đoán và giá trị thực tế đang là 47% khá là lớn cho thấy mô hình này đang bị underfitting, cần dùng các mô hình phức tạp hơn